In [1]:
import subprocess

import pandas as pd

from od_perturber import od_perturber
from output_reader import output_reader

# Automated OD trials

This is written to be run on a **Linux** machine.

In [2]:
def tap_trial(
            netFileName,
            demandFileName, 
            perturbType="normal", 
            norm_mean=1, 
            norm_sd=.1, 
            uniform_low=.9, 
            uniform_high=1.1,
            nodesPerturbedAlways=[],
            nodesPerturbedIfOrig=[],
            nodesPerturbedIfDest=[],
            returnDataFrame=False
):
    '''
    Generate a new TSTT calculation for a specific network, trip demands, and perturbation combination.
    '''
    od_perturber(
        demandFileName, 
        perturbType, 
        norm_mean, 
        norm_sd, 
        uniform_low, 
        uniform_high,
        nodesPerturbedAlways,
        nodesPerturbedIfOrig,
        nodesPerturbedIfDest
    )
    subprocess.Popen("tap-b/bin/tap " + netFileName + " trips_perturbed.tntp", shell=True).wait()
    if returnDataFrame:
        return output_reader("full_log.txt")
    else:
        _,tstt = output_reader("full_log.txt")
        return tstt

In [3]:
df, tstt = tap_trial("SiouxFalls_net.tntp", "SiouxFalls_trips.tntp", returnDataFrame=True)
tstt

7545042.096875504

In [4]:
df.head()

,OD,flow,cost,der
ID,,,,
0,"(1,2)",4426.697929,6.000768,0.000001
1,"(1,3)",8023.515936,4.008289,0.000004
2,"(2,1)",4703.696103,6.000979,0.000001
3,"(2,6)",5901.953126,6.505757,0.001021
4,"(3,1)",8037.612046,4.008347,0.000004


In [5]:
def tap_multi_trial(
            netFileName,
            demandFileName, 
            n, 
            perturbType="normal", 
            norm_mean=1, 
            norm_sd=.1, 
            uniform_low=.9, 
            uniform_high=1.1,
            nodesPerturbedAlways=[],
            nodesPerturbedIfOrig=[],
            nodesPerturbedIfDest=[]
):
    return [tap_trial(
        netFileName,
        demandFileName, 
        perturbType, 
        norm_mean, 
        norm_sd, 
        uniform_low, 
        uniform_high,
        nodesPerturbedAlways,
        nodesPerturbedIfOrig,
        nodesPerturbedIfDest,
        returnDataFrame=False
    ) for i in range(n)]

Every once in a while we get a TSTT of 0, resulting from a bug in the TAP-C code that identifies a cycle in the network file even though the network file hasn't changed from prior runs. 

In [6]:
tap_multi_trial("SiouxFalls_net.tntp", "SiouxFalls_trips.tntp", 10)

[7483625.665959306,
 7348116.258860868,
 7412376.119753626,
 7531381.102040663,
 7424215.439863765,
 7591657.92303383,
 7439751.077350472,
 7425978.413337935,
 7468471.217704278,
 7419492.0238108635]

## Generate basic trials on Sioux Falls

In [7]:
sds = [0.5, 0.1, 0.01, 0.001, 0]
means = [0.5, 0.9, 0.95, 1.0, 1.05, 1.1, 1.5, 2]
dfres = pd.DataFrame(columns=['mean', 'sd', 'tstt'])
i=0
for mean in means:
    for sd in sds:
        tstt = tap_trial("SiouxFalls_net.tntp", "SiouxFalls_trips.tntp", norm_mean=mean, norm_sd=sd, returnDataFrame=False)
        dfres.loc[i] = [mean, sd, tstt]
        i += 1
dfres.to_csv("res.csv")